In [1]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.sql.functions as F

# Initialize Spark
spark = SparkSession.builder \
    .appName("PracticeProject") \
    .enableHiveSupport() \
    .getOrCreate()

In [3]:
# Sample Data
customers_data = [
    (101, 'Ali', 'ali@gmail.com', 'Mumbai', '2022-05-10'),
    (102, 'Neha', 'neha@yahoo.com', 'Delhi', '2023-01-15'),
    (103, 'Ravi', 'ravi@hotmail.com', 'Bangalore', '2021-11-01'),
    (104, 'Sneha', 'sneha@outlook.com', 'Hyderabad', '2020-07-22'),
    (105, 'Amit', 'amit@gmail.com', 'Chennai', '2023-03-10'),
]

orders_data = [
    (1, 101, 'Laptop', 'Electronics', 2, 50000.0, '2024-01-10'),
    (2, 101, 'Mouse', 'Electronics', 1, 1200.0, '2024-01-15'),
    (3, 102, 'Tablet', 'Electronics', 1, 20000.0, '2024-02-01'),
    (4, 103, 'Bookshelf', 'Furniture', 1, 3500.0, '2024-02-10'),
    (5, 104, 'Mixer', 'Appliances', 1, 5000.0, '2024-02-15'),
    (6, 105, 'Notebook', 'Stationery', 5, 500.0, '2024-03-01'),
    (7, 102, 'Phone', 'Electronics', 1, 30000.0, '2024-03-02'),
]

# Create DataFrames
customers_df = spark.createDataFrame(customers_data, ["CustomerID", "Name", "Email", "City", "SignupDate"])
orders_df = spark.createDataFrame(orders_data, ["OrderID", "CustomerID", "Product", "Category", "Quantity", "Price", "OrderDate"])

# Create database and tables
spark.sql("CREATE DATABASE IF NOT EXISTS sales")
customers_df.write.mode("overwrite").saveAsTable("sales.customers")
orders_df.write.mode("overwrite").saveAsTable("sales.orders")

print("Data preparation complete!")
customers_df.show()
orders_df.show()

Data preparation complete!
+----------+-----+-----------------+---------+----------+
|CustomerID| Name|            Email|     City|SignupDate|
+----------+-----+-----------------+---------+----------+
|       101|  Ali|    ali@gmail.com|   Mumbai|2022-05-10|
|       102| Neha|   neha@yahoo.com|    Delhi|2023-01-15|
|       103| Ravi| ravi@hotmail.com|Bangalore|2021-11-01|
|       104|Sneha|sneha@outlook.com|Hyderabad|2020-07-22|
|       105| Amit|   amit@gmail.com|  Chennai|2023-03-10|
+----------+-----+-----------------+---------+----------+

+-------+----------+---------+-----------+--------+-------+----------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|
+-------+----------+---------+-----------+--------+-------+----------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|
|      2|       101|    Mouse|Electronics|       1| 1200.0|2024-01-15|
|      3|       102|   Tablet|Electronics|       1|20000.0|2024-02-01|
|      4|       103|Book

In [4]:
orders_df = orders_df.withColumn("TotalAmount", col("Price") * col("Quantity"))
print("Task 1 - Added TotalAmount column:")
orders_df.show()

Task 1 - Added TotalAmount column:
+-------+----------+---------+-----------+--------+-------+----------+-----------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|TotalAmount|
+-------+----------+---------+-----------+--------+-------+----------+-----------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|
|      2|       101|    Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|
|      3|       102|   Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|
|      4|       103|Bookshelf|  Furniture|       1| 3500.0|2024-02-10|     3500.0|
|      5|       104|    Mixer| Appliances|       1| 5000.0|2024-02-15|     5000.0|
|      6|       105| Notebook| Stationery|       5|  500.0|2024-03-01|     2500.0|
|      7|       102|    Phone|Electronics|       1|30000.0|2024-03-02|    30000.0|
+-------+----------+---------+-----------+--------+-------+----------+-----------+



In [5]:
high_value_orders = orders_df.filter(col("TotalAmount") > 10000)
print("\nTask 2 - Orders with TotalAmount > 10000:")
high_value_orders.show()


Task 2 - Orders with TotalAmount > 10000:
+-------+----------+-------+-----------+--------+-------+----------+-----------+
|OrderID|CustomerID|Product|   Category|Quantity|  Price| OrderDate|TotalAmount|
+-------+----------+-------+-----------+--------+-------+----------+-----------+
|      1|       101| Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|
|      3|       102| Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|
|      7|       102|  Phone|Electronics|       1|30000.0|2024-03-02|    30000.0|
+-------+----------+-------+-----------+--------+-------+----------+-----------+



In [6]:
customers_df = customers_df.withColumn("City", lower(col("City")))
print("\nTask 3 - Standardized City names:")
customers_df.show()


Task 3 - Standardized City names:
+----------+-----+-----------------+---------+----------+
|CustomerID| Name|            Email|     City|SignupDate|
+----------+-----+-----------------+---------+----------+
|       101|  Ali|    ali@gmail.com|   mumbai|2022-05-10|
|       102| Neha|   neha@yahoo.com|    delhi|2023-01-15|
|       103| Ravi| ravi@hotmail.com|bangalore|2021-11-01|
|       104|Sneha|sneha@outlook.com|hyderabad|2020-07-22|
|       105| Amit|   amit@gmail.com|  chennai|2023-03-10|
+----------+-----+-----------------+---------+----------+



In [7]:
orders_df = orders_df.withColumn("OrderYear", year(col("OrderDate")))
print("\nTask 4 - Added OrderYear column:")
orders_df.show()


Task 4 - Added OrderYear column:
+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|TotalAmount|OrderYear|
+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|     2024|
|      2|       101|    Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|     2024|
|      3|       102|   Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|     2024|
|      4|       103|Bookshelf|  Furniture|       1| 3500.0|2024-02-10|     3500.0|     2024|
|      5|       104|    Mixer| Appliances|       1| 5000.0|2024-02-15|     5000.0|     2024|
|      6|       105| Notebook| Stationery|       5|  500.0|2024-03-01|     2500.0|     2024|
|      7|       102|    Phone|Electronics|       1|30000.0|2024-03-02|    30000.0|     2024|
+-------+----------+---------+------

In [13]:
# Task 5 - Proper solution that works with the current orders_df structure
from pyspark.sql import Row

# Create a new row with ALL current columns (including the added ones)
# Note: Quantity is None to be filled later
new_row_data = [
    (8, 105, "Pen", "Stationery", None, 10.0, "2024-03-05", None, None)  # None for Quantity, TotalAmount and OrderYear
]

# Create with explicit schema matching current orders_df
new_row = spark.createDataFrame(new_row_data, orders_df.schema)

# Union the dataframes
orders_with_null = orders_df.union(new_row)

# First, fill the 'Quantity' null value with 1
orders_filled_quantity = orders_with_null.fillna({"Quantity": 1})

# Then, recalculate 'TotalAmount' and 'OrderYear' for all rows.
# This will correctly calculate values for the row where Quantity was filled.
filled_orders = orders_filled_quantity.withColumn("TotalAmount", col("Price") * col("Quantity")) \
                                      .withColumn("OrderYear", year(col("OrderDate")))


print("\nTask 5 - After adding null row and filling values:")
filled_orders.show()


Task 5 - After adding null row and filling values:
+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|TotalAmount|OrderYear|
+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|     2024|
|      2|       101|    Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|     2024|
|      3|       102|   Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|     2024|
|      4|       103|Bookshelf|  Furniture|       1| 3500.0|2024-02-10|     3500.0|     2024|
|      5|       104|    Mixer| Appliances|       1| 5000.0|2024-02-15|     5000.0|     2024|
|      6|       105| Notebook| Stationery|       5|  500.0|2024-03-01|     2500.0|     2024|
|      7|       102|    Phone|Electronics|       1|30000.0|2024-03-02|    30000.0|     2024|
|      8|       10

In [14]:
orders_df = orders_df.withColumn("OrderCategory",
    when(col("TotalAmount") < 5000, "Low")
    .when((col("TotalAmount") >= 5000) & (col("TotalAmount") <= 20000), "Medium")
    .otherwise("High"))
print("\nTask 6 - Categorized orders:")
orders_df.show()


Task 6 - Categorized orders:
+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+-------------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|TotalAmount|OrderYear|OrderCategory|
+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+-------------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|     2024|         High|
|      2|       101|    Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|     2024|          Low|
|      3|       102|   Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|     2024|       Medium|
|      4|       103|Bookshelf|  Furniture|       1| 3500.0|2024-02-10|     3500.0|     2024|          Low|
|      5|       104|    Mixer| Appliances|       1| 5000.0|2024-02-15|     5000.0|     2024|       Medium|
|      6|       105| Notebook| Stationery|       5|  500.0|2024-03-01|     2500.0|     2024|          Low|
|      

In [15]:
spark.sql("USE sales")
ali_orders = spark.sql("""
    SELECT o.*
    FROM orders o
    JOIN customers c ON o.CustomerID = c.CustomerID
    WHERE c.Name = 'Ali'
""")
print("\nTask 7 - Orders made by Ali:")
ali_orders.show()


Task 7 - Orders made by Ali:
+-------+----------+-------+-----------+--------+-------+----------+
|OrderID|CustomerID|Product|   Category|Quantity|  Price| OrderDate|
+-------+----------+-------+-----------+--------+-------+----------+
|      1|       101| Laptop|Electronics|       2|50000.0|2024-01-10|
|      2|       101|  Mouse|Electronics|       1| 1200.0|2024-01-15|
+-------+----------+-------+-----------+--------+-------+----------+



In [16]:
customer_spending = spark.sql("""
    SELECT c.CustomerID, c.Name, SUM(o.Price * o.Quantity) as TotalSpending
    FROM customers c
    JOIN orders o ON c.CustomerID = o.CustomerID
    GROUP BY c.CustomerID, c.Name
    ORDER BY TotalSpending DESC
""")
print("\nTask 8 - Total spending by each customer:")
customer_spending.show()


Task 8 - Total spending by each customer:
+----------+-----+-------------+
|CustomerID| Name|TotalSpending|
+----------+-----+-------------+
|       101|  Ali|     101200.0|
|       102| Neha|      50000.0|
|       104|Sneha|       5000.0|
|       103| Ravi|       3500.0|
|       105| Amit|       2500.0|
+----------+-----+-------------+



In [17]:
top_category = spark.sql("""
    SELECT Category, SUM(Price * Quantity) as TotalRevenue
    FROM orders
    GROUP BY Category
    ORDER BY TotalRevenue DESC
    LIMIT 1
""")
print("\nTask 9 - Category with highest total revenue:")
top_category.show()


Task 9 - Category with highest total revenue:
+-----------+------------+
|   Category|TotalRevenue|
+-----------+------------+
|Electronics|    151200.0|
+-----------+------------+



In [18]:
spark.sql("""
    CREATE OR REPLACE VIEW customer_orders AS
    SELECT c.Name as CustomerName, o.Product, (o.Price * o.Quantity) as TotalAmount
    FROM customers c
    JOIN orders o ON c.CustomerID = o.CustomerID
""")
print("\nTask 10 - Created customer_orders view. Sample data:")
spark.sql("SELECT * FROM customer_orders LIMIT 5").show()


Task 10 - Created customer_orders view. Sample data:
+------------+---------+-----------+
|CustomerName|  Product|TotalAmount|
+------------+---------+-----------+
|         Ali|   Laptop|   100000.0|
|         Ali|    Mouse|     1200.0|
|        Neha|   Tablet|    20000.0|
|        Ravi|Bookshelf|     3500.0|
|       Sneha|    Mixer|     5000.0|
+------------+---------+-----------+



In [19]:
post_feb_orders = spark.sql("""
    SELECT co.*, o.OrderDate
    FROM customer_orders co
    JOIN sales.orders o ON co.Product = o.Product
    WHERE o.OrderDate >= '2024-03-01'
""")
print("\nTask 11 - Products ordered after Feb 2024:")
post_feb_orders.show()


Task 11 - Products ordered after Feb 2024:
+------------+--------+-----------+----------+
|CustomerName| Product|TotalAmount| OrderDate|
+------------+--------+-----------+----------+
|        Amit|Notebook|     2500.0|2024-03-01|
|        Neha|   Phone|    30000.0|2024-03-02|
+------------+--------+-----------+----------+



In [20]:
customers_df.createGlobalTempView("customers")
print("\nTask 12 - Customers from Mumbai (Global Temp View):")
spark.sql("SELECT * FROM global_temp.customers WHERE City = 'mumbai'").show()


Task 12 - Customers from Mumbai (Global Temp View):
+----------+----+-------------+------+----------+
|CustomerID|Name|        Email|  City|SignupDate|
+----------+----+-------------+------+----------+
|       101| Ali|ali@gmail.com|mumbai|2022-05-10|
+----------+----+-------------+------+----------+



In [21]:
orders_df.write.mode("overwrite").parquet("orders_with_total.parquet")
print("\nTask 13 - Saved orders with TotalAmount to Parquet file")


Task 13 - Saved orders with TotalAmount to Parquet file


In [22]:
parquet_orders = spark.read.parquet("orders_with_total.parquet")
order_count = parquet_orders.count()
print(f"\nTask 14 - Number of orders in Parquet file: {order_count}")


Task 14 - Number of orders in Parquet file: 7


In [23]:
def mask_email(email):
    if not email or '@' not in email:
        return email
    name, domain = email.split('@', 1)
    return f"{name[0]}***@{domain}"

mask_email_udf = udf(mask_email, StringType())
customers_df = customers_df.withColumn("MaskedEmail", mask_email_udf(col("Email")))
print("\nTask 15 - Masked emails:")
customers_df.select("Email", "MaskedEmail").show()


Task 15 - Masked emails:
+-----------------+----------------+
|            Email|     MaskedEmail|
+-----------------+----------------+
|    ali@gmail.com|  a***@gmail.com|
|   neha@yahoo.com|  n***@yahoo.com|
| ravi@hotmail.com|r***@hotmail.com|
|sneha@outlook.com|s***@outlook.com|
|   amit@gmail.com|  a***@gmail.com|
+-----------------+----------------+



In [24]:
customers_df = customers_df.withColumn("FullLabel",
    concat_ws(" from ", col("Name"), col("City")))
print("\nTask 16 - Full labels:")
customers_df.select("FullLabel").show()


Task 16 - Full labels:
+--------------------+
|           FullLabel|
+--------------------+
|     Ali from mumbai|
|     Neha from delhi|
| Ravi from bangalore|
|Sneha from hyderabad|
|   Amit from chennai|
+--------------------+



In [25]:
orders_df = orders_df.withColumn("CleanProduct",
    regexp_replace(col("Product"), "[^a-zA-Z0-9]", ""))
print("\nTask 17 - Products with special characters removed:")
orders_df.select("Product", "CleanProduct").show()


Task 17 - Products with special characters removed:
+---------+------------+
|  Product|CleanProduct|
+---------+------------+
|   Laptop|      Laptop|
|    Mouse|       Mouse|
|   Tablet|      Tablet|
|Bookshelf|   Bookshelf|
|    Mixer|       Mixer|
| Notebook|    Notebook|
|    Phone|       Phone|
+---------+------------+



In [26]:
customers_df = customers_df.withColumn("SignupDate", to_date(col("SignupDate"))) \
    .withColumn("DaysSinceSignup",
        datediff(current_date(), col("SignupDate")))
print("\nTask 18 - Customer age in days:")
customers_df.select("Name", "SignupDate", "DaysSinceSignup").show()


Task 18 - Customer age in days:
+-----+----------+---------------+
| Name|SignupDate|DaysSinceSignup|
+-----+----------+---------------+
|  Ali|2022-05-10|           1121|
| Neha|2023-01-15|            871|
| Ravi|2021-11-01|           1311|
|Sneha|2020-07-22|           1778|
| Amit|2023-03-10|            817|
+-----+----------+---------------+

